Heatwave Index - New approach

In [1]:
import pandas as pd
import xarray as xr
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import math
import os

In [2]:
#Openfiles
path_t2m_calib = r"C:\Cataldi\Artigos\2024\Argel\Argelia\t2m_argel_1950_2023.nc"
ds_t2m_calib = xr.open_dataset(path_t2m_calib)
ds_t2m_calib["t2m"] = ds_t2m_calib.t2m - 273.15
#Training period
per = ["1960-01-01T00:00:00.000000000", "1990-12-31T23:00:00.000000000"]
ds_t2m_calib_period = ds_t2m_calib.sel(time = slice(per[0], per[1]))
#Daily maximum temperature
ds_t2m_calib_period = ds_t2m_calib_period.resample(time='D').max(dim='time')
#Monthly xarray datasets
month_ds = [ds_t2m_calib_period.sel(time=(ds_t2m_calib_period['time.month'] == i)) for i in range(1,13)] #Armazenando o dataset de cada mês na lista month_ds
#Monthly dataframes
lista_df = []
treinamento_percentil_95 = []
for i in range(0, 12):
    df = month_ds[i].to_dataframe()
    df = df.reset_index()
    df = df[["time", "t2m"]]
    lista_df.append(df)
    percentil_95 = np.percentile(month_ds[i]["t2m"], 95)
    treinamento_percentil_95.append(percentil_95)
    print(percentil_95, i+1)

for df in lista_df:
    df.set_index("time", inplace = True)
    
lista_df[0]

19.014007568359375 1
20.33417510986328 2
21.494293212890625 3
23.237744140625 4
25.786529541015625 5
29.518962097167968 6
33.227325439453125 7
33.483367919921875 8
31.662971496582028 9
28.606109619140625 10
24.359884643554686 11
20.0040283203125 12


,t2m
time,
1960-01-01,17.945526
1960-01-02,17.830109
1960-01-03,16.673370
1960-01-04,15.991974
1960-01-05,15.891083
...,...
1990-01-27,19.387726
1990-01-28,16.711395
1990-01-29,14.244476


In [3]:
#Prediction period
#Openfiles
path_t2m_prev = r"C:\Cataldi\Artigos\2024\Argel\Argelia\t2m_argel_1950_2023.nc"
path_r_prev = r"C:\Cataldi\Artigos\2024\Argel\Argelia\relum_argel_1950_2023.nc"

ds_r_prev = xr.open_dataset(path_r_prev)
ds_r_prev["r"] = ds_r_prev.r

ds_t2m_prev = xr.open_dataset(path_t2m_prev)
ds_t2m_prev["t2m"] = ds_t2m_prev.t2m - 273.15
ds_t2m_prev["r"] = ds_r_prev.r

#Prediction period
per_prev = ["1950-01-01T00:00:00.000000000", "2022-12-31T23:00:00.000000000"]
#per_prev = ["1960-01-01T00:00:00.000000000", "1990-12-31T23:00:00.000000000"]
ds_t2m_prev = ds_t2m_prev.sel(time = slice(per_prev[0], per_prev[1]))
month_ds_prev = [ds_t2m_prev.sel(time=(ds_t2m_prev['time.month'] == i)) for i in range(1,13)] #Armazenando o dataset de cada mês na lista month_ds_prev

#CRIAR AS SÉRIES TEMPORAIS PREVISÃO 
lista_df_prev = []

for i in range(0, 12):
    df_prev = month_ds_prev[i].to_dataframe()
    df_prev = df_prev.reset_index()
    df_prev = df_prev[["time", "t2m", "r"]]
    lista_df_prev.append(df_prev)

for df_prev in lista_df_prev:
    df_prev.set_index("time", inplace = True)
    df_prev = df_prev.sort_index()

lista_df_prev[0]

,t2m,r
time,,
1950-01-01 00:00:00,9.101318,84.972565
1950-01-01 01:00:00,8.656464,85.873154
1950-01-01 02:00:00,8.352295,86.179497
1950-01-01 03:00:00,8.433685,86.657959
1950-01-01 04:00:00,8.507111,86.820862
...,...,...
2022-01-31 19:00:00,14.510101,67.527145
2022-01-31 20:00:00,14.223541,68.397293
2022-01-31 21:00:00,13.025024,70.626282


In [4]:
#Calculation of Heatwave Index
for i in range(0, 12):
    #Extracting data from calibration's t2m
    temps = lista_df[i]['t2m'].values
    #Function for calculate the percentil of a x's temperature
    def calculate_percentil(x):
        return stats.percentileofscore(temps, x)
    
    # Aplicando a função de percentil para cada valor de temperatura no df2
    # Applying the function for each value in lista_df_prev[i]
    lista_df_prev[i]['Target'] = lista_df_prev[i]['t2m'].map(calculate_percentil)
    #Tme/tpe
    lista_df_prev[i]["tpe"] = lista_df_prev[i]["Target"] - 95
    lista_df_prev[i].loc[lista_df_prev[i]["Target"] - 95 <= 0, "tpe"] = 0
    
    #Coef
    lista_df_prev[i]["Coef"] = ((np.exp(lista_df_prev[i]["tpe"])) * (lista_df_prev[i]["r"])) / (1000)
    
    #Heatwave index
    lista_df_prev[i]["HWI"] = ((lista_df_prev[i]["Coef"]) - (0.022)) / (9.7)
    #Se tpe == 0, o índice não deve ser calculado
    lista_df_prev[i].loc[lista_df_prev[i]["tpe"] == 0, "HWI"] = 0
    #Se HWI <= 0.001, não contar como ondas de calor, portanto, HWI é transformado em zero.
    lista_df_prev[i].loc[lista_df_prev[i]["HWI"] <= 0.022, "HWI"] = 0

    #Saving the data in .csv
    result = r"C:\Cataldi\Artigos\2024\Argel\Argelia"
    if not os.path.exists(result):
        os.makedirs(result) 
    lista_df_prev[i].to_csv(f"{result}/heatwave_ERA5_month_{i+1}.csv")
    #lista_df_prev[i].to_excel(f"{result}/heatwave_ERA5_month_{i+1}.xlsx")

In [5]:
#testing the script
t2m_95 = lista_df_prev[9].loc[lista_df_prev[9]['Target'] >= 95]
t2m_95.sort_values(by="HWI")

,t2m,r,Target,tpe,Coef,HWI
time,,,,,,
2022-10-27 13:00:00,28.741241,15.862111,95.421436,0.421436,0.024176,0.000000
2013-10-04 10:00:00,29.024475,52.121727,96.045786,1.045786,0.148319,0.000000
2013-10-18 12:00:00,28.855255,46.018642,95.525494,0.525494,0.077831,0.000000
2013-10-22 09:00:00,28.618408,34.029709,95.005203,0.005203,0.034207,0.000000
2013-10-25 17:00:00,28.946747,17.334366,95.837669,0.837669,0.040059,0.000000
...,...,...,...,...,...,...
1981-10-05 11:00:00,33.283417,22.203772,99.895942,4.895942,2.969663,0.303883
1981-10-05 12:00:00,33.460144,21.026840,100.000000,5.000000,3.120660,0.319449
2001-10-12 11:00:00,33.208282,23.911974,99.895942,4.895942,3.198128,0.327436


Ind_prod Calculation

In [6]:
#Diary Ind_Prod
path1 = r"C:\Cataldi\Artigos\2024\Argel\Argelia"

lista_calib = []

for i in range(1, 13):
    path = f"{path1}/heatwave_ERA5_month_{i}.csv"
    df = pd.read_csv(path)
    df.reset_index()
    df.set_index("time", inplace=True)
    df.index = pd.to_datetime(df.index)
    
    # Grouping the data by day and calculating the sum of the index values
    soma_diaria = df.resample('D')['HWI'].sum()
    # Calculating the total number of hours the index is non-zero for each day
    horas_nao_zero = df['HWI'].apply(lambda x: 1 if x != 0 else 0).resample('D').sum()
    Ind_prod = soma_diaria * horas_nao_zero
    
    t2m = df.resample("D")["t2m"].mean()
    r = df.resample("D")["r"].mean()
    
    soma_diaria.name = 'diary_sum'
    horas_nao_zero.name = 'non_zero_hours'
    Ind_prod.name = 'Ind_prod'

    df_I = pd.concat([t2m, r, soma_diaria, horas_nao_zero, Ind_prod], axis=1)

    lista_calib.append(df_I)
    

#lista_calib[0]
Ind_Prod = pd.concat(lista_calib)
Ind_Prod.dropna(how="any", inplace=True)
Ind_Prod.index = pd.to_datetime(Ind_Prod.index)
Ind_Prod.sort_index(inplace=True)
Ind_Prod.to_csv("Diary_Ind_Prod.csv")
Ind_Prod

,t2m,r,diary_sum,non_zero_hours,Ind_prod
time,,,,,
1950-01-01,10.422157,84.956945,0.000000,0,0.000000
1950-01-02,10.975796,85.949195,0.000000,0,0.000000
1950-01-03,11.435977,69.444635,0.000000,0,0.000000
1950-01-04,12.857619,80.106444,0.000000,0,0.000000
1950-01-05,11.884970,77.955377,0.000000,0,0.000000
...,...,...,...,...,...
2022-12-27,14.928516,43.329172,0.000000,0,0.000000
2022-12-28,14.823129,35.785865,0.000000,0,0.000000
2022-12-29,14.263560,62.594521,0.000000,0,0.000000


In [7]:
#Monthly Ind_Prod
path1 = r"C:\Cataldi\Artigos\2024\Argel\Argelia"

lista_calib = []

for i in range(1, 13):
    path = f"{path1}/heatwave_ERA5_month_{i}.csv"
    df = pd.read_csv(path)
    df.reset_index()
    df.set_index("time", inplace=True)
    df.index = pd.to_datetime(df.index)
    
    # Grouping the data by day and calculating the sum of the index values
    soma_diaria = df.resample('D')['HWI'].sum()
    # Calculating the total number of hours the index is non-zero for each day
    horas_nao_zero = df['HWI'].apply(lambda x: 1 if x != 0 else 0).resample('D').sum()
    Ind_prod = soma_diaria * horas_nao_zero
    
    t2m = df.resample("D")["t2m"].mean()
    r = df.resample("D")["r"].mean()
    
    soma_diaria.name = 'diary_sum'
    horas_nao_zero.name = 'non_zero_hours'
    Ind_prod.name = 'Ind_prod'

    df_I = pd.concat([t2m, r, soma_diaria, horas_nao_zero, Ind_prod], axis=1)

    t2m = df_I.resample("MS")["t2m"].max()
    r = df_I.resample("MS")["r"].mean()
    Ind_prod_sum = df_I.resample('MS')['Ind_prod'].sum()
    Ind_prod_sum.name = "Ind_prod_monthly_sum"

    hwi = pd.concat([t2m, r, Ind_prod_sum], axis=1)
    hwi = hwi.dropna(how="any")

    lista_calib.append(hwi)
    

Ind_Prod = pd.concat(lista_calib)
Ind_Prod.index = pd.to_datetime(Ind_Prod.index)
Ind_Prod.sort_index(inplace=True)
Ind_Prod.to_csv("Monthly_Ind_Prod.csv")
Ind_Prod

,t2m,r,Ind_prod_monthly_sum
time,,,
1950-01-01,12.857619,74.169228,0.000000
1950-02-01,14.820431,65.629303,0.000000
1950-03-01,15.781166,70.751661,0.000000
1950-04-01,19.576688,66.545949,0.000000
1950-05-01,25.622873,67.093221,19.468229
...,...,...,...
2022-08-01,32.358966,56.118369,14.984372
2022-09-01,30.749914,63.911010,31.158931
2022-10-01,26.906530,54.615719,21.516153
